In [3]:
# Importing libraries
import tensorflow as tf
import keras as kr
import pandas as pd
import numpy as np



winningTimeMen = pd.read_csv("mensTime.csv", error_bad_lines=False) # reads data from mensTime
winningTimeWomen = pd.read_csv("womensTime.csv", error_bad_lines=False)

#print(type(winningTimeMen))  #DataFrame or dictionary for Men 
print(winningTimeWomen.shape)
print(winningTimeWomen.year)
print(type(winningTimeWomen))  #DataFrame or dictionary for Women

#print(winningTimeMen.mens) # a column of all the names for men
#print(winningTimeWomen.womens) # prints column of all names for women

resultsMen = np.polyfit(winningTimeMen.year, winningTimeMen.mens,1)
resultsWomen = np.polyfit(winningTimeWomen.year, winningTimeWomen.womens, 1) # fits model with years and womens, first power

m = resultsMen[0]
b = resultsMen[1]
mW = resultsWomen[0]
bW = resultsWomen[1]

print ("m= "+str(m)+" and b= "+str(b))

resultsCubicWomen = np.polyfit(winningTimeWomen.year, winningTimeWomen.womens,3)
resultsCubicMen = np.polyfit(winningTimeMen.year, winningTimeMen.mens,3)
m1Men = resultsCubicMen[0]
m2Men = resultsCubicMen[1]
m3Men = resultsCubicMen[2]
m4Men = resultsCubicMen[3]

m1Women = resultsCubicWomen[0]
m2Women = resultsCubicWomen[1]
m3Women = resultsCubicWomen[2]
m4Women = resultsCubicWomen[3]

resultsEightMen = np.polyfit(winningTimeMen.year, winningTimeMen.mens,8)
resultsEightWomen = np.polyfit(winningTimeWomen.year, winningTimeWomen.womens, 8) # fits model with years and womens, 8th power
resultsSquareMen = np.polyfit(winningTimeMen.year, winningTimeMen.mens, 2)
resultsSquareWomen = np.polyfit(winningTimeWomen.year, winningTimeWomen.womens, 2)
#new_model = tf.saved_model.load("saved_model")


# Alternative method
modelMen = kr.Sequential([kr.layers.Dense(units=1, input_shape= [1,])])
modelMen.compile(optimizer='sgd', loss="mean_squared_error", metrics=["accuracy"])

modelWomen = kr.Sequential([kr.layers.Dense(units=1, input_shape= [1,])])
modelWomen.compile(optimizer='sgd', loss="mean_squared_error", metrics=["accuracy"])

minimumYearMen = winningTimeMen.year.min()
maximumYearMen = winningTimeMen.year.max()

minimumYearWomen = winningTimeWomen.year.min()
maximumYearWomen = winningTimeWomen.year.max()

#print(minimumYear, maximumYear)
xsM = winningTimeMen.year - minimumYearMen
xsW = winningTimeWomen.year - minimumYearWomen
#print("after subtracting min")
#print(xs)

xsM /= (maximumYearMen - minimumYearMen) #we normalize the x's so it is nicely spread between 0 and 1.
xsW /= (maximumYearWomen - minimumYearWomen) #we normalize the x's so it is nicely spread between 0 and 1.
ysM = winningTimeMen.mens
ysW = winningTimeWomen.womens

sum = 0.0

#print("now xs normalized")

modelMen.fit(xsM, ysM, epochs=500)
modelWomen.fit(xsW, ysW, epochs=500)

print ("now the prediction ... ")
#print (model([0.9861]))
#predictions = new_model.predict(xs, verbose=0)
#model = tf.saved_model.load("the_saved_model") 
predictionsMen = modelMen.predict(xsM, verbose=1)
predictionsWomen = modelWomen.predict(xsW, verbose=1)
print("the model, men: ")
#print (new_model.weights)
print (modelMen.predict([0.39, 0.73, 0.9861, 1.14], verbose=0))
tf.saved_model.save(modelMen, "the_saved_model_men")
print("the model, women: ")
print (modelWomen.predict([0.39, 0.73, 0.9861, 1.14], verbose=0))
tf.saved_model.save(modelWomen, "the_saved_model_women")

import numpy as np
import matplotlib.pyplot as plt

xM = xsM
xW = xsW
yM = ysM
yW = ysW

# now x is back to normal.
xM = 112 * xsM + 1896
xW = 80 * xsW + 1928

# Plotting men
figMen = plt.figure()
plt.plot(xM, m * xM + b,'b-') # blue line : Statistics linear model
plt.plot(xM,m1Men*xM*xM*xM+m2Men*xM*xM+m3Men*xM+m4Men,'g-')
plt.plot(xM, yM,'bo') # data points with blue dots
plt.plot(xM,predictionsMen,'r-') # red line : AI model, 2500 iterations. 
columnToAdd = [1.0]*len(xM)

# Plotting women
#figWomen = plt.figure()
#plt.plot(xW, m * xW + b,'b-') # blue line : Statistics linear model
#plt.plot(xW,m1Women*xW*xW*xW+m2Women*xW*xW+m3Women*xW+m4Women,'g-')
#plt.plot(xW, yW,'bo') # data points with blue dots
#plt.plot(xW,predictionsWomen,'r-') # red line : AI model, 2500 iterations. 
#columnToAdd = [1.0]*len(xW)



plt.plot(xM, m * xM + b,'b-') # blue line : Statistics linear model
plt.plot(xM,m1Men*xM*xM*xM+m2Men*xM*xM+m3Men*xM+m4Men,'g-')
plt.plot(xM, yM,'bo') # data points with blue dots
plt.plot(xM,predictionsMen,'r-') # red line : AI model, 2500 iterations. 
columnToAddMen = [1.0]*len(xM)
#columnToAddWomen = [1.0]*len(xW)
matriXM = pd.DataFrame()
#matriXW = pd.DataFrame() # how do you define 
degree = 8
for i in range(0,degree):
  matriXM.insert(i, column=i, value=xM**(8-i))

matriXM.insert(8, column=8, value = columnToAddMen)
matrixForMultMen = matriXM.to_numpy()
print(resultsEightWomen)
print(resultsEightMen)
yEight=np.matmul(matrixForMultMen,resultsEightMen)
ySquare = np.matmul(matrixForMultMen, resultsSquareMen)
print(yEightMen)
plt.plot(xM,yEightMen,'-')

fig.savefig("overfit.jpg")


(19, 2)
0     1928
1     1932
2     1936
3     1948
4     1952
5     1956
6     1960
7     1964
8     1968
9     1972
10    1976
11    1980
12    1984
13    1988
14    1992
15    1996
16    2000
17    2004
18    2008
Name: year, dtype: int64
<class 'pandas.core.frame.DataFrame'>
m= -0.013330885710959656 and b= 36.41645590250102
Epoch 1/500
1/1 [==============================] - 0s 2ms/step - loss: 108.4339 - accuracy: 0.0000e+00
Epoch 2/500
1/1 [==============================] - 0s 1ms/step - loss: 103.1385 - accuracy: 0.0000e+00
Epoch 3/500
1/1 [==============================] - 0s 1ms/step - loss: 98.1092 - accuracy: 0.0000e+00
Epoch 4/500
1/1 [==============================] - 0s 1000us/step - loss: 93.3324 - accuracy: 0.0000e+00
Epoch 5/500
1/1 [==============================] - 0s 999us/step - loss: 88.7956 - accuracy: 0.0000e+00
Epoch 6/500
1/1 [==============================] - 0s 1ms/step - loss: 84.4866 - accuracy: 0.0000e+00
Epoch 7/500
1/1 [==============================] - 

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 9)

In [ ]:
msg = "Hello world"
print(msg)

# New Section